### same as the article on the website, but using averagepooling instead of max pooling

- very slow. stopped after 2 epochs

In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D, GlobalAvgPool1D
from keras.models import Model

from helpers import files

Using TensorFlow backend.


In [2]:
texts,labels_index,labels = files.read_20_newgroup_files()       

In [3]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

In [4]:
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
X_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [5]:
embeddings_index = files.read_glove_100()

In [6]:
len(word_index)

174074

In [7]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False)

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

x = Conv1D(MAX_SEQUENCE_LENGTH, MAX_SEQUENCE_LENGTH, activation='relu')(embedded_sequences)
x = GlobalAvgPool1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

In [ ]:
model = Model(sequence_input, preds)
model.compile(loss ='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['acc'])

In [ ]:
%%time

model.fit(X_train,y_train, validation_data=(X_val, y_val),
         epochs=10, batch_size=32)